# Nearest Neighbor Experiment

This notebook will use the dataset prepared by another notebook [prepare_dataset](prepare_dataset.ipynb).

In [5]:
import numpy as np
import pandas as pd
import fasttext.util
from sklearn.neighbors import NearestNeighbors
import nagisa

-----
## Load the dataset

We will load the prepared csv file, this csv included a field `concat_name`, which is the concatenated name + postal code + address.


In [7]:
df = pd.read_csv("./features_full.csv")

In [9]:
df.sample(5)

,法人名,concat_name
547367,有限会社笠幡,有限会社笠幡 埼玉県鶴ヶ島市大字鶴ヶ丘 733
3988461,株式会社ライフエイト,株式会社ライフエイト 東京都新宿区歌舞伎町 2-8-3
3663394,大正埠頭作業株式会社,大正埠頭作業株式会社 大阪府大阪市大正区小林西 1-25-13
4279765,有限会社中山商事,有限会社中山商事 東京都文京区本郷 1-20-3
2516069,合資会社ホルモン栄養研究所洋酒部,合資会社ホルモン栄養研究所洋酒部 大阪府大阪市天王寺区東平野町一丁目 27


In [10]:
features = df["concat_name"]
company_names = df["法人名"]

Save some memory..

In [11]:
del(df)

-----
## Prepare JP tokenizer


In [12]:
# Takes in a document, filtering out particles, punctuation, and verb endings
def tokenize_jp(text):
    # text = text.
    doc = nagisa.filter(text, filter_postags=['空白', '助詞', '補助記号', '助動詞'])
    return doc.words

print(features[0])
print(tokenize_jp(features[0]))
print(features[1])
print(tokenize_jp(features[1]))

釧路検察審査会 北海道釧路市柏木町 4-7 
['釧路', '検察', '審査', '会', '北海道', '釧路', '市', '柏木町', '4', '7']
伊達簡易裁判所 北海道伊達市末永町 47-10 
['伊達', '簡易', '裁判', '所', '北海道', '伊達', '市', '末永', '町', '4', '7', '1', '0']


-----
## Load the Embedding converter

We will use the same embedding from fast Text. This embedding will be used during inference to convert the input text.

In [42]:
# Load the pre-trained model
ft = fasttext.load_model('cc.ja.300.bin')

Test its feature

In [44]:
ft.get_word_vector("有限会社PIA 東京都千代田区内神田 2-11-4 トーハンビル5階")[:10]

array([ 0.00068098, -0.0003277 ,  0.00181239, -0.00983721, -0.00108764,
        0.00500185, -0.00861101, -0.00744833, -0.00053406,  0.00237652],
      dtype=float32)

--------
## Exp. A:  Nearest neighbor model (with fastText Embedding)

In [45]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(features_vec)

NearestNeighbors(metric='cosine', n_neighbors=10)

Now we have the model prepared, let's try

In [69]:
input_texts = [
    # 有限会社PIA 東京都千代田区内神田 2-11-4 トーハンビル5階
    "有限 P I A 東京都千代田区内神田 2-11-4",
    # 鹿高神社 三重県名張市安部田 1942 
    "三重県名張市安部田",
]
input_features = [ft.get_word_vector(x) for x in input_texts]

# get the result
D, N = knn.kneighbors(input_features, n_neighbors=5, return_distance=True)

for input_text, distances, neighbors in zip(input_texts, D, N):
    print("Input text = ", input_text[:200], "\n")
    for dist, neighbor_idx in zip(distances, neighbors):
        print(f"Distance = {dist:0.4f}", ", Index = ", neighbor_idx, " , target text: ", features[neighbor_idx])
    print("="*200)
    print()


Input text =  有限 P I A 東京都千代田区内神田 2-11-4 

Distance = 0.3833 , Index =  375  , target text:  アンフィニィ株式会社 東京都千代田区内神田 2-13-14 
Distance = 0.4221 , Index =  756  , target text:  オーストラリア政府観光局 東京都千代田区永田町 2-11-1 山王パークタワー12階
Distance = 0.4265 , Index =  406  , target text:  合同会社ブリーズ 兵庫県神戸市東灘区本山中町 4-10-20 
Distance = 0.4268 , Index =  185  , target text:  社会復帰サポート美祢株式会社 東京都渋谷区神宮前 1-5-1 
Distance = 0.4320 , Index =  285  , target text:  株式会社ライフイノベーション 神奈川県横浜市港北区新横浜 3-13-6 新横浜葉山第3ビル3階

Input text =  三重県名張市安部田 

Distance = 0.5078 , Index =  2  , target text:  鹿高神社 三重県名張市安部田 1942 
Distance = 0.5209 , Index =  752  , target text:  マークスター有限会社 福岡県福岡市早良区干隈 4-25-1-404 
Distance = 0.5559 , Index =  641  , target text:  氷川神社 埼玉県上尾市大字平塚 1514 
Distance = 0.5587 , Index =  104  , target text:  合資会社三共商會 山口県下関市大字関後地村 1032 
Distance = 0.5622 , Index =  160  , target text:  桂昌庵 島根県雲南市吉田町吉田 2367 



From the above example, seems the model cannot search the name if the input missed or provide wrong info.

I suspect the reason is that the text embedding we used are focusing in the context too much. But we only need to focus at the "wordings" in this situation.


-----

## Try with other text vectors - TF-IDF

By the above observation, let's try TF-IDF.

In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(tokenizer=tokenize_jp, ngram_range=(1,5))

We will fit the TFIDF model with the text features.

In [138]:

features_tfidf = vec.fit_transform(features)
print(features.shape)
print(features_tfidf.shape)

/Users/leung.tsz.kit/Desktop/work/code/test/github_repos/ann_search_sample/annoy/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1000,)
(1000, 40089)


Then we can train the nearest neighbor model with this vectors

In [139]:
knn_tfidf = NearestNeighbors(n_neighbors=10, metric='cosine')
knn_tfidf.fit(features_tfidf)

NearestNeighbors(metric='cosine', n_neighbors=10)

Now we have the model ready, let's give it a try.

In [141]:
input_texts = [
    # 有限会社PIA 東京都千代田区内神田 2-11-4 トーハンビル5階
    " P IA 式 ト 東京都千代田区内 2-1-4 6階",
    # 鹿高神社 三重県名張市安部田 1942 
    "鹿高神社三重県名張市安部1942",
]
input_features = vec.transform(input_texts)

# get the result
D, N = knn_tfidf.kneighbors(input_features, n_neighbors=5, return_distance=True)

for input_text, distances, neighbors in zip(input_texts, D, N):
    print("Input text = ", input_text[:200], "\n")
    for dist, neighbor_idx in zip(distances, neighbors):
        print(f"Distance = {dist:0.4f}", ", Index = ", neighbor_idx, " , target text: ", features[neighbor_idx])
    print("="*200)
    print()


Input text =   P IA 式 ト 東京都千代田区内 2-1-4 6階 

Distance = 0.6345 , Index =  375  , target text:  アンフィニィ株式会社 東京都千代田区内神田 2-13-14 
Distance = 0.6903 , Index =  3  , target text:  有限会社PIA 東京都千代田区内神田 2-11-4 トーハンビル5階
Distance = 0.8372 , Index =  253  , target text:  デュリソルアジア株式会社 東京都千代田区永田町 2-9-8 
Distance = 0.8401 , Index =  162  , target text:  株式会社ラクーン 東京都千代田区外神田 5-2-10 
Distance = 0.8405 , Index =  146  , target text:  有限会社マイセン 東京都千代田区神田小川町 1-11 

Input text =  鹿高神社三重県名張市安部1942 

Distance = 0.4454 , Index =  2  , target text:  鹿高神社 三重県名張市安部田 1942 
Distance = 0.8310 , Index =  334  , target text:  八阪神社 三重県四日市市赤堀 2-9-8 
Distance = 0.9092 , Index =  447  , target text:  株式会社北清緑建 宮崎県都城市太郎坊町 942-1 
Distance = 0.9140 , Index =  646  , target text:  一般社団法人天原会 福岡県春日市大土居 1-94 
Distance = 0.9161 , Index =  31  , target text:  株式会社パネルヤ 千葉県印旛郡栄町安食台 3-19-4 



Still, we can see the second example  "鹿高神社三重県名張市安部1942", it is strange that cannot match just due to extra space.

------
## Try with Bag of words




In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Have to use JP tokenizer because default one does not know how to split words
vectorizer = CountVectorizer(tokenizer=tokenize_jp, ngram_range=(1,5))

In [14]:
# Fit the bag of word model
features_bow = vectorizer.fit_transform(features)

/Users/leung.tsz.kit/Desktop/work/code/test/github_repos/ann_search_sample/annoy/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

Let's visualize how vectorizer split the address:

In [131]:
analyze_bow = vectorizer.build_analyzer()
print(features[0])
print(analyze_bow(features[0]))

株式会社新潟丸和運輸 新潟県新潟市東区海老ケ瀬字長田 502-1 
['株式', '会社', '新潟', '丸和', '運輸', '新潟', '県', '新潟', '市', '東', '区', '海老ケ瀬', '字', '長田', '5', '0', '2', '1', '株式 会社', '会社 新潟', '新潟 丸和', '丸和 運輸', '運輸 新潟', '新潟 県', '県 新潟', '新潟 市', '市 東', '東 区', '区 海老ケ瀬', '海老ケ瀬 字', '字 長田', '長田 5', '5 0', '0 2', '2 1', '株式 会社 新潟', '会社 新潟 丸和', '新潟 丸和 運輸', '丸和 運輸 新潟', '運輸 新潟 県', '新潟 県 新潟', '県 新潟 市', '新潟 市 東', '市 東 区', '東 区 海老ケ瀬', '区 海老ケ瀬 字', '海老ケ瀬 字 長田', '字 長田 5', '長田 5 0', '5 0 2', '0 2 1', '株式 会社 新潟 丸和', '会社 新潟 丸和 運輸', '新潟 丸和 運輸 新潟', '丸和 運輸 新潟 県', '運輸 新潟 県 新潟', '新潟 県 新潟 市', '県 新潟 市 東', '新潟 市 東 区', '市 東 区 海老ケ瀬', '東 区 海老ケ瀬 字', '区 海老ケ瀬 字 長田', '海老ケ瀬 字 長田 5', '字 長田 5 0', '長田 5 0 2', '5 0 2 1', '株式 会社 新潟 丸和 運輸', '会社 新潟 丸和 運輸 新潟', '新潟 丸和 運輸 新潟 県', '丸和 運輸 新潟 県 新潟', '運輸 新潟 県 新潟 市', '新潟 県 新潟 市 東', '県 新潟 市 東 区', '新潟 市 東 区 海老ケ瀬', '市 東 区 海老ケ瀬 字', '東 区 海老ケ瀬 字 長田', '区 海老ケ瀬 字 長田 5', '海老ケ瀬 字 長田 5 0', '字 長田 5 0 2', '長田 5 0 2 1']


Or we can look at the terms stored in the model directly:

In [132]:
vectorizer.get_feature_names_out()

array(['0', '0 0', '0 0 0', ..., '𣳾 新 商事', '𣳾 新 商事 北海道', '𣳾 新 商事 北海道 札幌'],
      dtype=object)

### Train Nearest Neighbor

In [6]:
knn_bow = NearestNeighbors(n_neighbors=10, metric='cosine')
knn_bow.fit(features_bow)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [7]:
input_texts = [
    # 有限会社PIA 東京都千代田区内神田 2-11-4 トーハンビル5階
    # "PIA",
    # "有限 P I A 式 ト 東京都千代田区内 2-1-4 56階",

    # 鹿高神社 三重県名張市安部田 1942 
    "高神社三重県名張市安部1942",
    "三重県名張市1942",

]
input_features = vectorizer.transform(input_texts)

# get the result
D, N = knn_bow.kneighbors(input_features, n_neighbors=5, return_distance=True)

for input_text, distances, neighbors in zip(input_texts, D, N):
    print("Input text = ", input_text[:200], "\n")
    for dist, neighbor_idx in zip(distances, neighbors):
        print(f"Distance = {dist:0.4f}", ", Index = ", neighbor_idx, " , target text: ", features[neighbor_idx])
    print("="*200)
    print()


Input text =  高神社三重県名張市安部1942 

Distance = 0.2546 , Index =  2  , target text:  鹿高神社 三重県名張市安部田 1942 
Distance = 0.7226 , Index =  334  , target text:  八阪神社 三重県四日市市赤堀 2-9-8 
Distance = 0.7750 , Index =  871  , target text:  有限会社マチノ 三重県四日市市久保田 2-14-4 
Distance = 0.7767 , Index =  447  , target text:  株式会社北清緑建 宮崎県都城市太郎坊町 942-1 
Distance = 0.7778 , Index =  443  , target text:  有限会社イエンプロダクション 三重県四日市市小生町 229-222 

Input text =  三重県名張市1942 

Distance = 0.3494 , Index =  2  , target text:  鹿高神社 三重県名張市安部田 1942 
Distance = 0.7545 , Index =  871  , target text:  有限会社マチノ 三重県四日市市久保田 2-14-4 
Distance = 0.7564 , Index =  447  , target text:  株式会社北清緑建 宮崎県都城市太郎坊町 942-1 
Distance = 0.7575 , Index =  443  , target text:  有限会社イエンプロダクション 三重県四日市市小生町 229-222 
Distance = 0.7685 , Index =  587  , target text:  合資会社田村製瓦工場 愛媛県今治市菊間町浜甲 1949 

